Script Purpose: Scrape cafes' reviews from Google Map

Coding Reference: https://www.linkedin.com/pulse/webscrape-google-map-reviews-using-selenium-python-choy-siew-wearn/?trk=pulse-article_more-articles_related-content-card

List of cafes:
- Close to the beaches (Queen St E.)
    - Bud's Coffee
    - Juice & Java Cafe
    - The Beacher Cafe
    - Simple Coffee
    - Remarkable Beans
    - The Haven Low-Carb Cafe
- Financial District (Bay St.)
    - Dineen Coffee Co.
    - Mos Mos Coffee
    - NEO COFFEE BAR
    - Coffee Island
    - White Rabbit Caffe
    - Dispatch Coffee


In [1]:
import pandas as pd
import time

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

In [15]:
# Create the function for getting reviews through url

def getReviews(cafe_name,street_name, url):
    import time 
    
    # Initialize WebDriver
    driver = webdriver.Chrome()

    # Specify the name here
    resturant_name = cafe_name

    # Replace the URL here
    url = url
    print(url)
    driver.get(url)

    #time.sleep(5)
    # Get the total number of reviews
    num_reviews = driver.find_element("xpath","//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]").text
    num_reviews = int(num_reviews.split(' ')[0].replace(",",''))

    # Click Sort button and select "Newest"
    # Page Source - Inspect - Locate the button and copy XPATH
    driver.find_element("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button').click()
    time.sleep(1)

    driver.find_element("xpath","//*[@id='action-menu']/div[2]").click()
    time.sleep(5)

    # Scoll the review pages from top to bottom
    SCROLL_PAUSE_TIME = 5
    number = 0

    while True:
        number = number+1

        # Scroll down to bottom    
        ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
        driver.execute_script('arguments[0].scrollBy(0, 5000);', ele)

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        if number >= num_reviews/10:
            break

    # Store all reviews elements
    item = driver.find_elements("xpath",'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]')
    time.sleep(3)

    # Initialize dataframe for reviews
    review_df = pd.DataFrame(index=range(1000),columns=['cafe','street','user','time','stars','reviews'])
    review_df['cafe'] = cafe_name
    review_df['street'] = street_name
    
    idx = 0
    for i in item:
        button = i.find_elements(By.TAG_NAME,'button')
        for m in button:
            if m.text == "More":
                m.click()
        time.sleep(5)

        name = i.find_elements(By.CLASS_NAME,"d4r55")
        time = i.find_elements(By.CLASS_NAME,"kvMYJc")
        stars = i.find_elements(By.CLASS_NAME,"wiI7pd")
        reviews = i.find_elements(By.CLASS_NAME,"rsqaWe")

        for j,k,l,p in zip(name,time,stars,reviews):
            review_df['user'][idx] = j.text
            review_df['time'][idx] = p.text
            review_df['stars'][idx] = k.get_attribute("aria-label")
            review_df['reviews'][idx] = l.text
        
            idx = idx + 1
        

    review_df = review_df.dropna(thresh=4)
    review_df.to_csv('google_review_{}.csv'.format(resturant_name),index=False)


    return review_df

In [3]:
# Read the list of cafes and their reviews URL
urls = pd.read_excel('CafesURL.xlsx')
urls.head()

,Cafe,Street,Review URL
0,Bud's Coffee,Queen St E,https://www.google.ca/maps/place/Bud's+Coffee/...
1,Juice & Java Cafe,Queen St E,https://www.google.ca/maps/place/Juice+%26+Jav...
2,The Beacher Cafe,Queen St E,https://www.google.ca/maps/place/The+Beacher+C...
3,Simple Coffee,Queen St E,https://www.google.ca/maps/place/Simple+Coffee...
4,Remarkable Beans,Queen St E,https://www.google.ca/maps/place/Remarkable+Be...


In [ ]:
#merged_df = pd.DataFrame()

for i in range(len(urls)):
    cafe_name = urls['Cafe'][i]
    cafe_street = urls['Street'][i]
    cafe_url = urls['Review URL'][i]

    df = getReviews(cafe_name=cafe_name, street_name=cafe_street, url=cafe_url)

    merged_df = pd.concat([merged_df,df], ignore_index=True)
    merged_df.head()

In [18]:
#merged_df = merged_df.dropna(thresh=4)
merged_df.tail()

,cafe,street,user,time,stars,reviews
3256,Dispatch Coffee,Bay St,Gabriel Lessard,a year ago,5 stars,Really good flat white. But what I loved the m...
3257,Dispatch Coffee,Bay St,Adam Gilani,a year ago,5 stars,Thank you Ally - see you soon :)
3258,Dispatch Coffee,Bay St,G,a year ago,1 star,
3259,Dispatch Coffee,Bay St,N'Gatta Aka,a year ago,5 stars,
3260,Dispatch Coffee,Bay St,Erinna Sanders,a year ago,5 stars,I do appreciate places like these. Designed fo...


In [19]:
merged_df.head()

,cafe,street,user,time,stars,reviews
0,Bud's Coffee,Queen St E,David Baker,6 days ago,4 stars,
1,Bud's Coffee,Queen St E,Michael Bunga,a week ago,5 stars,
2,Bud's Coffee,Queen St E,Jeffrey Taylor,2 weeks ago,5 stars,Great coffee and fresh baked goods.\nTalking w...
3,Bud's Coffee,Queen St E,Dasong Zou,2 weeks ago,5 stars,I had a great tea here. Very pleasant.
4,Bud's Coffee,Queen St E,Carrie T,4 weeks ago,4 stars,


In [22]:
merged_df.to_csv('cafes_reviews.csv', index = False)

In [ ]:
import os 
import glob

path = os.getcwd()
csv_files = glob.glob(os.path.join(path,"*.csv"))

merged_df = pd.DataFrame()
for f in csv_files:
    df = pd.read_csv(f)
    df['Cafe'] = f.split("\\")[-1].split('_')[-1].split('.')[0]

    print('Location:', f)
    print('File Name:', f.split("\\")[-1])

    merged_df = pd.concat([merged_df,df], ignore_index=True)

Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\cafes_reviews.csv
File Name: cafes_reviews.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Bud's Coffee.csv
File Name: google_review_Bud's Coffee.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Coffee Island.csv
File Name: google_review_Coffee Island.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Dineen Coffee Co.csv
File Name: google_review_Dineen Coffee Co.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Dispatch Coffee.csv
File Name: google_review_Dispatch Coffee.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturant-reviews-analysis\google_review_Juice & Java Cafe.csv
File Name: google_review_Juice & Java Cafe.csv
Location: c:\Users\shans\OneDrive\Desktop\GitHub\google-resturan